<a href="https://colab.research.google.com/github/johnflem/IBM-Final-Model/blob/main/dash%20example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Import required libraries
!pip install pandas
!pip install dash
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output

import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd




--2024-03-29 15:16:35--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2476 (2.4K) [text/csv]
Saving to: ‘spacex_launch_dash.csv.1’

spacex_launch_dash. 100%[===================>]   2.42K  --.-KB/s    in 0s      

2024-03-29 15:16:35 (1.05 GB/s) - ‘spacex_launch_dash.csv.1’ saved [2476/2476]

--2024-03-29 15:16:35--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-objec

In [10]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")

max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

app = dash.Dash(__name__)

# Define the application layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard'),
    dcc.Dropdown(
        id='site-dropdown',
        options=[{'label': 'All Sites', 'value': 'ALL'}] +
                 [{'label': i, 'value': i} for i in spacex_df['Launch Site'].unique()],
        value='ALL',
        placeholder="Select a launch site here",
        searchable=True
    ),

    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),
    html.P("Payload range (Kg):"),

    dcc.RangeSlider(
        id='payload-slider',
        min=min_payload,
        max=max_payload,
        step=1000,  # Adjust this value as needed
        marks={i: '{}'.format(i) for i in range(int(min_payload), int(max_payload)+1, 10000)},
        value=[min_payload, max_payload]
    ),

    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

# TASK 2 Callback: Update Pie Chart
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value')
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        fig_data = spacex_df.groupby('Launch Site')['class'].sum().reset_index()
        fig = go.Figure(data=[go.Pie(labels=fig_data['Launch Site'], values=fig_data['class'])])
    else:
        fig_data = spacex_df[spacex_df['Launch Site'] == selected_site]['class'].value_counts().reset_index()
        fig = go.Figure(data=[go.Pie(labels=fig_data['index'].map({1: 'Success', 0: 'Failure'}), values=fig_data['class'])])
    fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20)
    return fig

# TASK 4 Callback: Update Scatter Chart
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('payload-slider', 'value'), Input('site-dropdown', 'value')]
)
def update_scatter_chart(payload_range, selected_site):
    # Ensure payload_range is a tuple/list with 2 elements



    if not payload_range or len(payload_range) != 2:
        return go.Figure()

    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload_range[0]) &
                            (spacex_df['Payload Mass (kg)'] <= payload_range[1])]



    if selected_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]

    # Handling empty filtered_df
    if filtered_df.empty:
        return {
            'data': [],
            'layout': go.Layout(title="No Data Available")
        }

    fig = go.Figure(data=go.Scatter(
        x=filtered_df['Payload Mass (kg)'],
        y=filtered_df['class'],
        mode='markers',
        marker=dict(
            color=filtered_df['class'],  # Confirm this is numeric or appropriately mapped
            colorscale='Viridis',
            size=10),
        text=filtered_df['class']))  # Ensure Mission Outcome exists and is suitable for display


#    fig.update_layout(
#        xaxis_title='Payload Mass (kg)',
#        yaxis_title='Launch Outcome',
#        title='Correlation between Payload and Success for all Launch Sites')

    return fig







if __name__ == '__main__':
    app.run_server(debug=True)




<IPython.core.display.Javascript object>